# Day 11: Space Police
https://adventofcode.com/2019/day/11

## Part 1

In [ ]:
import numpy as np
import urllib.request
import math
import itertools

Int machine from prevoious days:

In [ ]:
class IntcodeParameter():
    POSITION = 0
    IMMEDIATE = 1
    RELATIVE = 2
    
    value = None
    mode = None
    
    def __init__(self, value, mode):
        self.value = value
        self.mode = mode
        assert mode in [IntcodeParameter.POSITION, IntcodeParameter.IMMEDIATE, IntcodeParameter.RELATIVE], f'UNKNOWN PARAMETER MODE {mode}'

    def checkBoundedParameter(self, program):
        if self.mode == IntcodeParameter.POSITION:
            assert self.value >= 0 and self.value < len(program.code), f'position parameter out of bounds. Position: {self.value}, Program size {len(program.code)}'
        elif self.mode == IntcodeParameter.RELATIVE:
            realPos = self.value + program.relative_base
            assert realPos >= 0 and realPos < len(program.code), f'relative parameter out of bounds. Position: {self.value}, Relative base {program.relative_base}, Program size {len(program.code)}'
        return True
        
    def evaluateParameter(self, program):
        if self.mode == IntcodeParameter.POSITION:
            return program.code[self.value]
        elif self.mode == IntcodeParameter.IMMEDIATE:
            return self.value
        elif self.mode == IntcodeParameter.RELATIVE:
            return program.code[self.value + program.relative_base]
        else:
            raise Exception('UNKNOWN PARAMETER MODE {}. Value {}. Program {}'.format(self.mode, self.value, program))
    
    def getParameterPosition(self, program):
        if self.mode == IntcodeParameter.POSITION:
            return self.value
        elif self.mode == IntcodeParameter.RELATIVE:
            return self.value + program.relative_base
        elif self.mode == IntcodeParameter.IMMEDIATE:
            raise Exception(f'Immediate parameter mode {self.value} cannot be used as position!')
        else:
            raise Exception(f'UNKNOWN PARAMETER MODE {self.mode}. Value {self.value}. Program {program}')
    
    def toString(self, program):
        return '[value {}, mode {}, evaluation {}]'.format(self.value, self.mode, self.evaluateParameter(program))

In [ ]:
class IntcodeInstruction():
    endComputations = False
    parameters = []
    
    def mustEndComputations(self):
        return self.endComputations
    
    def getInstructionSize(self):
        return len(self.parameters) + 1 #parameters + opcode
    
    def getNextInstructionPointer(self, program):
        newIP = program.IP + self.getInstructionSize()
        assert newIP >= 0 and newIP < len(program.code), f'new instruction pointer out of bounds. New IP {newIP}, last IP {program.IP}, Program size {len(program.code)}'
        return newIP
    
    def extractParameters(self, program):
        pass
    
    def performComputation(self, program):
        pass
    
    def checkParameters(self, program):
        for param in self.parameters:
            param.checkBoundedParameter(program)
    
    def loadParameters(self, program):
        self.parameters = self.extractParameters(program)
        self.checkParameters(program)
    
    def compute(self, program):
        self.performComputation(program)
    
    def toString(self, program):
        msg = f'{self.__class__.__name__} '
        for param in self.parameters:
            msg += param.toString(program)
        
        return (msg)

class OneParameterInst(IntcodeInstruction):
    def extractParameters(self, program):
        parameters = []
        modes = program.code[program.IP] // 100
        parameters.append(IntcodeParameter(program.code[program.IP + 1], modes % 10))
        
        return parameters

class TwoParametersInst(OneParameterInst):
    def extractParameters(self, program):
        parameters = super().extractParameters(program)
        modes = program.code[program.IP] // 100
        #1 parameters + 1 result
        modes //= 10
        parameters.append(IntcodeParameter(program.code[program.IP + 2], modes % 10))
        
        return parameters
    
class ThreeParameterInst(TwoParametersInst):
    def extractParameters(self, program):
        parameters = super().extractParameters(program)
        modes = program.code[program.IP] // 100
        #2 parameters + 1 result
        modes //= 10
        modes //= 10
        parameters.append(IntcodeParameter(program.code[program.IP + 3], modes % 10))
        
        return parameters
    
class AdditionInst(ThreeParameterInst):
    def performComputation(self, program):
        resultValue = self.parameters[0].evaluateParameter(program) + self.parameters[1].evaluateParameter(program)
        program.code[self.parameters[2].getParameterPosition(program)] = resultValue
        
class MultiplyInst(ThreeParameterInst):
    def performComputation(self, program):
        resultValue = self.parameters[0].evaluateParameter(program) * self.parameters[1].evaluateParameter(program)
        program.code[self.parameters[2].getParameterPosition(program)] = resultValue

class LessThanInst(ThreeParameterInst):
    def performComputation(self, program):
        resultValue = 0
        if self.parameters[0].evaluateParameter(program) < self.parameters[1].evaluateParameter(program):
            resultValue = 1
        program.code[self.parameters[2].getParameterPosition(program)] = resultValue
        
class EqualsInst(ThreeParameterInst):
    def performComputation(self, program):
        resultValue = 0
        if self.parameters[0].evaluateParameter(program) == self.parameters[1].evaluateParameter(program):
            resultValue = 1
        program.code[self.parameters[2].getParameterPosition(program)] = resultValue
        
        
class InputInst(OneParameterInst):
    def performComputation(self, program):
        n = input('Solicitando dato de entrada:')
        
        # TODO: Validar dato!!! Debe ser entero, dado que luego podrá ser
        # usado para operar con él o ser un opcode, etc..
        
        program.code[self.parameters[0].getParameterPosition(program)] = int(n)

class OutputInst(OneParameterInst):
    def performComputation(self, program):
        print('>>>OUTPUT: {}'.format(self.parameters[0].evaluateParameter(program)))

class JumpIfTrueInst(TwoParametersInst):
    def performComputation(self, program):
#         print('JumpIfTrueInst!')
        pass

    def getNextInstructionPointer(self, program):
        if self.parameters[0].evaluateParameter(program) != 0:
            return self.parameters[1].evaluateParameter(program)
        
        return super().getNextInstructionPointer(program)

class JumpIfFalseInst(TwoParametersInst):
    def performComputation(self, program):
#         print('JumpIfFalseInst!')
        pass

    def getNextInstructionPointer(self, program):
        if self.parameters[0].evaluateParameter(program) == 0:
            return self.parameters[1].evaluateParameter(program)
        
        return super().getNextInstructionPointer(program)

class AdjustRelativeBaseInst(OneParameterInst):
    def performComputation(self, program):
        shift = self.parameters[0].evaluateParameter(program)
        program.relative_base += shift
    
class QuitInst(IntcodeInstruction):
    def __init__(self):
        self.endComputations = True

    def extractParameters(self, program):
        return []
        
    def performComputation(self, program):
        pass

In [ ]:
class IntcodeProgram():
    MAX_SIZE_IMAGE = 2000
    IP = None
    code = None
    relative_base = None
    
    def __init__(self, code, relative_base = 0):
        self.IP = 0
        self.relative_base = relative_base
        self.code = self.__expand_code(code) #maybe code.copy() ?

    def __expand_code(self, code):
        assert len(code) < IntcodeProgram.MAX_SIZE_IMAGE, f'Program is too big! program size {len(code)}. Max size {IntcodeProgram.MAX_SIZE_IMAGE}'
        return code + ( (IntcodeProgram.MAX_SIZE_IMAGE - len(code)) * [0]  )

In [ ]:
class IntcodeInterpreterV4():
    instructions = {}
    program = None
    
    def __init__(self):
        # TODO: Inject instruction set
        # Arithmetic
        self.instructions[1]  = AdditionInst()
        self.instructions[2]  = MultiplyInst()
        
        # IO
        self.instructions[3]  = InputInst()
        self.instructions[4]  = OutputInst()
        
        #Logic
        self.instructions[7]  = LessThanInst()
        self.instructions[8]  = EqualsInst()
        
        #Jump
        self.instructions[5]  = JumpIfTrueInst()
        self.instructions[6]  = JumpIfFalseInst()

        #Mem
        self.instructions[9]  = AdjustRelativeBaseInst()
        
        #STOP
        self.instructions[99] = QuitInst()

    def getInstruction(self, program):
        opcode = program.code[program.IP]
        opcode = opcode % 100
        if opcode in self.instructions:
            instruction = self.instructions[opcode]
            instruction.loadParameters(program)
            return instruction
        else:
            raise Exception('UNKNOWN opcode {} ({}) at position {}.\nPROGRAM: {}'.format(opcode, program.code[program.IP], program.IP, program.code))
        return None
        
    def compute(self, ext_program):
#         print(f'Executing {ext_program}!')
        self.program = IntcodeProgram(ext_program)
        while True:
            # Extract next instruction            
            instruction = self.getInstruction(self.program)
                
            # Exit if it must
            if instruction.mustEndComputations():
                break
            else:
                # Otherwise, compute instruction
                instruction.compute(self.program)
                # Change IP (Instruction Pointer)
                self.program.IP = instruction.getNextInstructionPointer(self.program)
            
#             print('DEBUGGGGGG')
#             print(program)
#             print('IP {}'.format(self.IP))
#             print('DEBUGGGGGG')
            
        return self.program.code

Utility functions from previous days:

In [ ]:
def checkMapIsValid(string_map):
    #Map must have one line, at least
    if len(string_map) == 0:
        return False
    #First line must have columns
    width = len(string_map[0])
    if width == 0:
        return False
    
    #All lines must have same length
    for num_line in range(1, len(string_map)):
        if len(string_map[num_line]) != width:
            return False
    return True

DIRECTION_UP = '^'
DIRECTION_RIGHT = '>'
DIRECTION_DOWN = 'v'
DIRECTION_LEFT = '<'

def loadMap(string_map):
    assert checkMapIsValid(string_map), 'Map is not valid!'
    
    height = len(string_map)
    width  = len(string_map[0])
    
    #Initializing zero-matrix. 
    #Setting 1 if white
    #Assuming the robot starts on a black tile
    int_map = np.zeros([height, width])
    starting_position = None
    starting_direction = None
    directions = [ DIRECTION_UP, DIRECTION_DOWN, DIRECTION_LEFT, DIRECTION_RIGHT ]
    for num_line in range(height):
        line = string_map[num_line]
        for num_col in range(width):
            symbol = line[num_col:num_col+1]
            if symbol == '#':
                int_map[num_line][num_col] = 1
            elif symbol in directions:
                starting_direction = symbol
                starting_position = (num_line, num_col)
    
    return int_map, starting_position, starting_direction

### Making the Robot

We'll create a robot with a IntcodeInterpreter, an screen, and a camera. IntcodeInterpreter will use input instructions that operate the camera, to see the color and output instructions to display on the screen color and directions.

#### Devices



In [ ]:
class Device():
    def read(self):
        raise Exception('Read operation not defined for this device!!')
        
    def write(self, data):
        raise Exception('Write operation not defined for this device!!')
    
class CameraDevice(Device):
    int_map = None
    attached_to = None
    
    def attachTo(self, robot):
        self.attached_to = robot

    def setMap(self, int_map):
        self.int_map = int_map
        
    def read(self):
        assert self.attached_to is not None, "Camera is not attached to anything!!"
        assert self.int_map is not None, "Map is not set!! Nothing to look at!!"
        
        (line, col) = self.attached_to.getCoordinates()
        value = self.int_map[line][col]
        
        print('[CAMERA]', f'({line},{col}) -> {value}')
        
        return value

class ScreenDevice(Device):
    buffer = None
    
    def __init__(self):
        self.buffer = []
        
    def write(self, data):
        print('[SCREEN]', data)
        self.buffer.append(data)
        
    def flushData(self):
        returnBuffer = self.buffer.copy()
        self.buffer = []
        return returnBuffer

#### New IO intcode instructions

In [ ]:
class IOInputInst(OneParameterInst):
    device = None
    def __init__(self, device):
        self.device = device
        
    def performComputation(self, program):
        assert self.device is not None, "Can't read from device: No device present!"
        n = self.device.read()
        
        # TODO: Validate data!!! It must be an int because it can be used later as an operator, an opcode...
        program.code[self.parameters[0].getParameterPosition(program)] = int(n)

class IOOutputInst(OneParameterInst):
    device = None
    def __init__(self, device):
        self.device = device
        
    def performComputation(self, program):
        assert self.device is not None, "Can't write to device: No device present!"
        
        data = self.parameters[0].evaluateParameter(program)
        
        self.device.write(data)

Depending of how this turns out, we'll create a "console device" to get data with python input sentence and output data with the python print, and always use these IO int code instructions later on. Also, we could code them to get/put data on the console if no device is present, by default.

#### The robot

In [ ]:
class PaintRobot():
    #constants
    UP = 0
    RIGHT = 1
    DOWN = 2
    LEFT = 3
    
    TURN_TO_LEFT = -1
    TURN_TO_RIGHT = 1
    
    brain = None
    camera = None
    screen = None
    col = None
    line = None
    direction = None
    
    program = None
    
    def __init__(self, brain, camera, screen):
        self.brain = brain
        self.screen = screen
        self.camera = camera
        
        # Init and connect components
        self.camera.attachTo(self)
        self.brain.instructions[3] = IOInputInst(self.camera)
        self.brain.instructions[4] = IOOutputInst(self.screen)
    
    def setDirection(self, direction):
        assert direction in range(0, 3), 'direction should be in [0,3] range!!'
        self.direction = direction
    
    def setCoordinates(self, coordinates):
        (self.line, self.col) = coordinates
    
    def getCoordinates(self):
        return (self.line, self.col)
    
    def turn(self, turn_to):
        assert turn_to == PaintRobot.TURN_TO_LEFT or turn_to == PaintRobot.TURN_TO_RIGHT, 'Incorrect turn direction!!'
        self.direction = (self.direction + turn_to) % 3
    
    def walk(self):
        step_size = 1
        if self.direction == PaintRobot.UP:
            self.line -= step_size
        elif self.direction == PaintRobot.DOWN:
            self.line += step_size
        elif self.direction == PaintRobot.LEFT:
            self.col -= step_size
        elif self.direction == PaintRobot.RIGHT:
            self.col += step_size
        else:
            raise Exception(f'Robot was looking at {self.direction}. Unknown direction!')
    
    def getScreenData(self):
        return self.screen.flushData()
    
    def startTask(self, program):
        self.program = program

    def nextStep(self):
        #Durante la ejecución se consulta la cámara
        print('nextStep', 'BEGIN STEP COMPUTATION')
        self.program = self.brain.compute(self.program)
        print('nextStep', 'END STEP COMPUTATION')
        #Aqui debería estar modificado ya el screen del robot
        #Cogemos el color y la nueva dirección
        
    
    def endTask(self):
        #Aquí deberíamos devolver los resultados
        pass
        

In [ ]:
def getRobotDirection(string_direction):
    if string_direction == DIRECTION_UP:
        return PaintRobot.UP
    elif string_direction == DIRECTION_RIGHT:
        return PaintRobot.RIGHT
    elif string_direction == DIRECTION_DOWN:
        return PaintRobot.DOWN
    elif string_direction == DIRECTION_LEFT:
        return PaintRobot.LEFT
    else:
        raise Exception(f'Direction {string_direction} is unkown!!')
    
def paintSourface(robot, code, string_map):
    int_map, starting_position, starting_direction = loadMap(string_map)
    robot.setDirection(getRobotDirection(starting_direction))
    robot.setCoordinates(starting_position)
    robot.camera.setMap(int_map)
    
    #TODO: check if starting_position is bounded by sourface limits
    robot.startTask(code)
    while robot.nextStep():
        pass
    #Aquí se supone que ya lo ha pintado todo.
    


In [ ]:
robot = PaintRobot(IntcodeInterpreterV4(), CameraDevice(), ScreenDevice())

In [ ]:
string_map = ['.....',
'.....',
'..^..',
'.....',
'.....']

In [ ]:
input_11 = r'data\aoc2019-input-day11.txt'
with open(input_11, 'r') as f:
    data11 = [int(data) for data in f.read().split(',') if len(data) > 0]

In [ ]:
paintSourface(robot, data11, string_map)

Turns out all the painting process occurs in just one machine computation!